In [1]:
import tweepy
import requests
import urllib
import json

In [2]:
from credentials import *

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
user = api.me()

In [4]:
def get_info(status):
    print("responding to: ")
    print(status.id)
    return {
        "text": status.text,
        "username": status.author.screen_name,
        "user_id": status.author.id,
        "in_reply_to": status.in_reply_to_status_id,
        "id": status.id   
    }

In [5]:
def is_int(string):
    try:
        print(string)
        return int(string)
    except ValueError:
        return False
def is_badgeNumber(string):
    return is_int(string) and len(string) > 3
def parse_StringForInt(string):
    words = string.split()
    for i in range(0, len(words)):
        word = words[i]
        #check for if suffixed by period
        if word[len(word) - 1] == ".":
            if is_int(word.split(".")[0]):
                return int(word.split(".")[0])
            else: 
                continue
        else:
            if is_int(word):
                return int(word)
    return None

In [6]:
print(is_int("5"))
print(parse_StringForInt("5."))

5
5
5
5


In [7]:
def refNumber(_id):
    try:
        tweet = api.get_status(_id)
    except Error:
        return None
    parts = get_info(tweet)["text"].split("ref:")
    print(parts)
    if len(parts) != 2:
        print("not two")
        return None
    else:
        if is_int(parts[1]):
            print("Is int")
            return is_int(parts[1])
        else:
            print("isnt int")
            return None

In [8]:
def respondToTweet(ref, status, copsDict):
    print("Getting info")
    tweet = "@{0} \n".format(status["username"])
    text = status["text"]
    urlquery = ""
    if (is_badgeNumber(text)):
        urlquery = urllib.parse.urlencode({
            "badge_number": text
        })
    else:
        parsedInt = parse_StringForInt(text)
        if (parsedInt != None):
            ref = refNumber(status["in_reply_to"])
            print(ref)
            if ref ==  None:
                print("ref none")
                return
            cops = copsDict[ref]
            if cops != None and parsedInt - 1 < len(cops):
                cop = cops[parsedInt - 1]
                copTweet = tweet + "Name: {0}\nBadge Number: {1}\nComplaints: {2}".format(cop["name"], cop["badge_number"], cop["complaint_count"])
                api.update_status(copTweet, status["id"])
                return
            else:
                return
        else:
            queryParts = status["text"].split()
            query = queryParts[1:len(queryParts)]
            queryText = ""
            if len(query) == 1:
                queryText = query[0]
            elif len(query) == 2:
                queryText = query[0] + " " + query[1]
            elif len(query) == 3:
                queryText = query[0] + " " + query[1] + " " + query[2]
            urlquery = urllib.parse.urlencode({
                "name": queryText
            })
    print(urlquery)
    r = requests.get("http://localhost:3000/api/v1/cops?{0}".format(urlquery) + "&limit=10")
    responseJson = r.json()
    cops = responseJson["cops"]
    if len(responseJson["cops"]) == 0:
        tweet += "We could not find any cops in the NYPD under the given search term."
        api.update_status(tweet, status["id"]) 
        return
    else:
        lengthTweet = tweet + "We found {0} cops that may match you're search. Respond with the number that you think is the cop you're looking for.".format(len(responseJson["cops"]))
        copsDict[ref] = cops
        api.update_status(lengthTweet, status["id"])
        for i in range(0, len(cops)):
            cop = responseJson["cops"][i]
            copTweet = tweet + "{0}. Name: {1}\nBadge Number: {2} \nref:{3}".format(i + 1, cop["name"], cop["badge_number"], ref)
            api.update_status(copTweet, status["id"])  
        ref += 1

In [11]:
class MyStreamListener(tweepy.StreamListener):
    copsDict = {}
    refNumber = 1
    def on_status(self, status):
        if status.user.id != user.id:
            info = get_info(status)
            respondToTweet(self.refNumber, info, self.copsDict)

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(follow=[user.id_str])

responding to: 
1384891236448878596
Getting info
@SatireChatBot String
@SatireChatBot
String
name=String
responding to: 
1384891309526290435
Getting info
@SatireChatBot 1
@SatireChatBot
1
responding to: 
1384891263837638662
['@CalvinC59692666 \n1. Name: Denise Stringer\nBadge Number: 6357 \n', '1']
1
Is int
1
1
